In [73]:
data = [
    ('education', '名师指导托福语法技巧：名词的复数形式'),
    ('education', '中国高考成绩海外认可 是“狼来了”吗？'),
    ('sports', '图文：法网孟菲尔斯苦战进16强 孟菲尔斯怒吼'),
    ('sports', '四川丹棱举行全国长距登山挑战赛 近万人参与'),
    ('sports', '米兰客场8战不败国米10年连胜')
]
data_m = data * 10
import pandas as pd

data_df = pd.DataFrame(data_m, columns=['label', 'text'])

In [84]:
data_df.label.head()

0    education
1    education
2       sports
3       sports
4       sports
Name: label, dtype: object

In [83]:
import jieba
def seg(x):
    return ' '.join(jieba.lcut(x))
data_df['text_seg'] = data_df['text'].map(seg)
data_df.head()

,label,text,text_seg
0,education,名师指导托福语法技巧：名词的复数形式,名师 指导 托福 语法 技巧 ： 名词 的 复数 形式
1,education,中国高考成绩海外认可 是“狼来了”吗？,中国 高考 成绩 海外 认可 是 “ 狼来了 ” 吗 ？
2,sports,图文：法网孟菲尔斯苦战进16强 孟菲尔斯怒吼,图文 ： 法网 孟菲尔 斯 苦战 进 16 强 孟菲尔 斯 怒吼
3,sports,四川丹棱举行全国长距登山挑战赛 近万人参与,四川 丹棱 举行 全国 长距 登山 挑战赛 近万人 参与
4,sports,米兰客场8战不败国米10年连胜,米兰 客场 8 战 不败 国米 10 年 连胜


In [78]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

vec = TfidfVectorizer(analyzer='word')
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)

pipe.fit(data_df.text_seg, data_df.label)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregressioncv', LogisticRegressionCV())])

In [79]:

from sklearn import metrics

def print_report(pipe):
    y_test =  data_df.label
    y_pred = pipe.predict(data_df.text_seg)
    report = metrics.classification_report(y_test, y_pred,
        target_names=['education','sports'])
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe)

              precision    recall  f1-score   support

   education       1.00      1.00      1.00        20
      sports       1.00      1.00      1.00        30

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

accuracy: 1.000


In [80]:
import eli5
eli5.show_weights(clf, top=10)

In [85]:
eli5.show_weights(clf, vec=vec,
                  target_names=['edu','sports'])

In [97]:
valid = ' '.join(jieba.lcut('中国福建春季公务员考试报名18日截止 2月6日考试'))
          
from eli5 import show_prediction
show_prediction(clf, valid, vec=vec, 
                show_feature_values=True, top=10,target_names=['sports','edu'])

In [98]:
from pytextclassifier import TextClassifier
import numpy as np

from sklearn import metrics
m = TextClassifier(model_name='lr')
data = m.load_file_data('./train_seg_sample.txt', '\t')
data.head()


[   INFO 20210903 01:57:10 textclassifier:  44] load file done. path: ./train_seg_sample.txt, size: 200


,label,text
0,财经,中新网 5月 9日 中国经济时报 报道 房价 五一 铺天盖地 口水战 理会 各方 面的 压力...
1,财经,nbsp&nbsp&nbsp&nbsp4 28日 氧化铝 供求 缺口 短期 很难 得到解决 ...
2,IT,法国 海关 没收 中国游客 假名牌 旅行包 处罚 55 中国 网友 欧洲 国家保护 知识产权...
3,健康,西班牙 欧华 中新网 5月 8日 马德里 警方 日前 fuenlabrada 大仓 库区 监...
4,财经,5月 09日 消息 深度 报告 权威 内参 证券 www.kl178.com 今日热点 五一...


In [104]:
b = []
for i in data.values:
    b.append((i[0], i[1]))

In [106]:
m.train(b)

[  DEBUG 20210903 01:59:12 textclassifier:  74] train model
[  DEBUG 20210903 01:59:12 textclassifier:  60] load data_list, X size: 200, label size: 200
[  DEBUG 20210903 01:59:12 textclassifier:  62] num_classes:3
[  DEBUG 20210903 01:59:13 textclassifier:  65] data tokens top 3: ['中新网 5 月 9 日 中国经济时报 报道 房价 五一 铺天盖地 口水战 理会 各方 面 的 压力 质疑 仍然 在 市场 供需 规则 向上 有人 提出 警告 房价 暴涨 再次出现 房地产 泡沫 真成 开发商 们 就 会 搬起 石头 砸 了 没有 可能 月底 第一份 商品房 空置率 增高 引发 房价 上涨 的 信息 披露 深圳 市民 成 了 焦点人物 网上 发表 公开信 掀起 买房 运动 到 了 全国 各地 人士 网上 支持 现象 背后 揭示 道理 那 就是 老百姓 面对 新房 无力 购买 一段 购买力 时期 消费 需求 时期 老百姓 承受 高房价 压力 房地产商 承受 资金 回笼 高额 贷款 利息 压力 这是 一把 双刃剑 东南 律师 事务所 主任 律师 接受 中国经济时报 记者 采访 4 月 广州 富力 地产 股份 有限公司 6 . 16 亿元 5 . 2 亿元 独揽 广州 珠江 新城 两块 商用 地块 楼面地价 升 到 每平方米 4308 元 下 了 珠江 新城 四年 拍卖 楼面地价 新高 就 在 一年 前 2005 年 3 月 珠江 新城 住宅 楼面地价 最高价 3348 元 平方米 一位 广东 房地产业 人士 记者 担忧 广州 珠江 新城 整体 售价 在短期内 提升 巨资 下 地 房地产商 企业 面临 生存 危机 记者 规划 建设 中 广州 cbd 发现 建 好 在售 商品 房价格 均 在 万元 平方米 开盘 楼价 已经 在 酝酿 着 开盘 均价 万元 平方米 珠江 新城 整体 开发 建设 高级别 商务 房价 涨幅 广州 可谓 史无前例 许多 人 担忧 珠江 新城 到 时 未能 成像 上海浦东 北京 

LogisticRegression(fit_intercept=False)

In [107]:
eli5.show_weights(m.model, vec=m.vectorizer,
                  target_names=['fin','it','health'])